# 提示词工程核心原则实践

基于吴恩达 x OpenAI 提示词工程课程的实践 notebook

## 📚 学习目标

通过精心设计的对比实验，深入理解两大核心原则：

1. **写清楚、具体的 Prompt**（4个策略）
2. **给模型时间思考**（2个策略）

## 🎯 实验设计理念

每个策略包含：

- ✅ **正例**：严格遵循原则，引导模型产生高质量输出
- ❌ **反例**：故意违反原则，暴露常见错误导致的问题

**关键**：反例不是随意设计，而是精心构造的陷阱，展示违反原则的真实后果。

## 🤖 测试模型

使用 **Qwen3 8B** 模型（Q4_K_M 量化，参数量充足，适合在 T4 GPU 上实验）

---

## 第一步：环境准备

安装并启动 Ollama 服务

In [17]:
# === 1. 在 Colab 安装并启动 Ollama ===
import subprocess
import time
import os

print("⬇️ 正在安装 Ollama...")
install_cmd = "curl -fsSL https://ollama.com/install.sh | sh"
install_result = subprocess.run(install_cmd, shell=True, check=False)
if install_result.returncode == 0:
    print("✅ Ollama 安装完成")
else:
    print("⚠️ Ollama 安装遇到问题，继续尝试...")
    try:
        subprocess.run(["apt", "update"], check=False)
        subprocess.run(["apt", "install", "-y", "curl"], check=False)
        install_result = subprocess.run(install_cmd, shell=True, check=False)
        if install_result.returncode == 0:
            print("✅ Ollama 安装完成（重试）")
        else:
            print("⚠️ Ollama 可能已安装或安装失败，继续执行...")
    except Exception as e:
        print(f"⚠️ 备用安装方式失败: {str(e)}，继续执行...")

print("🚀 正在后台启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已在 localhost:11434 运行")

⬇️ 正在安装 Ollama...
✅ Ollama 安装完成
🚀 正在后台启动 Ollama 服务...
✅ Ollama 服务已在 localhost:11434 运行


## 第二步：下载模型

下载 Qwen3 8B 模型（推荐在 T4 GPU 上使用）

In [2]:
# === 2. 下载 Qwen3 8B 模型 ===
import subprocess

print("⬇️ 正在下载 qwen3:8b...")
print("💡 提示：使用 Q4_K_M 量化，约 5.2GB，适合 T4 GPU")
pull_result = subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
if pull_result.returncode != 0:
    raise RuntimeError("❌ qwen3:8b 下载失败")

print("✅ 模型下载完成！")

⬇️ 正在下载 qwen3:8b...
💡 提示：使用 Q4_K_M 量化，约 5.2GB，适合 T4 GPU
✅ 模型下载完成！


## 第三步：导入依赖并定义核心函数

定义与模型交互的函数

In [3]:
# === 3. 导入依赖并定义核心函数 ===
import requests
import json
import time

OLLAMA_URL = "http://localhost:11434"

def get_completion(prompt, model="qwen3:8b", temperature=0.7):
    """调用 Qwen3 模型获取回复"""
    url = f"{OLLAMA_URL}/api/chat"

    options = {
        "temperature": temperature,
        "top_p": 0.9,
        "num_ctx": 8192,
        "num_predict": 4096
    }

    messages = [{"role": "user", "content": prompt}]
    payload = {"model": model, "messages": messages, "stream": True, "options": options}

    try:
        response = requests.post(url, json=payload, stream=True)
        response.raise_for_status()

        result = ""
        print("🤖 模型回复: ", end='', flush=True)

        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    if 'message' in data and 'content' in data['message']:
                        content = data['message']['content']
                        if content:
                            print(content, end='', flush=True)
                            result += content
                    if data.get('done', False):
                        print()
                        break
                except json.JSONDecodeError:
                    continue
        return result.strip()
    except Exception as e:
        print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]"

def compare_prompts(good_prompt, bad_prompt, title, show_prompts=True):
    """对比好的提示词和坏的提示词的效果"""
    print("\n" + "="*80)
    print(f"🔬 {title}")
    print("="*80)

    # 测试正例
    print("\n✅ 正例：遵循原则的提示词")
    print("─"*80)
    if show_prompts:
        print(f"提示词:\n{good_prompt}\n")
        print("─"*80)
    good_response = get_completion(good_prompt)
    print(f"\n📊 输出长度: {len(good_response)} 字符")

    print("\n" + "─"*80)
    time.sleep(2)

    # 测试反例
    print("\n❌ 反例：违反原则的提示词")
    print("─"*80)
    if show_prompts:
        print(f"提示词:\n{bad_prompt}\n")
        print("─"*80)
    bad_response = get_completion(bad_prompt)
    print(f"\n📊 输出长度: {len(bad_response)} 字符")

    # 对比分析
    print("\n" + "="*80)
    print("📈 对比分析")
    print("="*80)
    print(f"正例输出长度: {len(good_response)} 字符")
    print(f"反例输出长度: {len(bad_response)} 字符")
    print("="*80 + "\n")

    return good_response, bad_response

print("✅ 核心函数加载完成")
print("💡 可用函数: get_completion(prompt), compare_prompts(good, bad, title)")

✅ 核心函数加载完成
💡 可用函数: get_completion(prompt), compare_prompts(good, bad, title)


---

# 原则一：写清楚、具体的 Prompt

## 策略 1.1：使用分隔符明确区分指令和内容

### 📖 原理说明

使用分隔符（如三重引号、XML 标签等）可以：

- **防止提示词注入**：用户输入的内容不会被误认为是给模型的指令
- **清晰的边界**：让模型明确知道「这是要处理的数据」vs「这是处理指令」
- **提高可靠性**：特别是当内容中包含指令性语言时

### 🎯 实验设计

**任务**：总结一段文字

**关键设计**：文本中故意包含容易混淆的指令性语言（\"忽略\"、\"请注意\"等）

- ✅ **正例**：用三重反引号或者xml标签包裹文本，清晰分隔
- ❌ **反例**：不用分隔符，让指令和内容混在一起

**预期差异**：反例可能被文本中的\"忽略\"、\"请注意\"等词汇误导

In [18]:
# 精心设计的测试文本：包含容易被误解为指令的语句
text_to_summarize = """提示词工程是一门新兴技术。请注意：很多人误以为提示词越短越好，
但这是错误的观点。忽略那些建议你写超短提示词的说法。
实际上，清晰详细的提示词才能获得更好的结果。
记住：写清楚、具体的指令是核心原则，这能显著减少AI的错误回答。"""

# ✅ 正例：使用分隔符明确边界
good_prompt_1_1 = f"""请将由<text>标签包裹的文本总结成一句话（不超过30字）。

文本：
<text>{text_to_summarize}</text>

总结："""

# ❌ 反例：不使用分隔符，容易混淆
bad_prompt_1_1 = f"""请将下面这段文本总结成一句话（不超过30字）。
{text_to_summarize}
总结："""

# 运行对比实验
compare_prompts(
    good_prompt_1_1,
    bad_prompt_1_1,
    "策略 1.1：使用分隔符明确区分指令和内容"
)


🔬 策略 1.1：使用分隔符明确区分指令和内容

✅ 正例：遵循原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
请将由<text>标签包裹的文本总结成一句话（不超过30字）。

文本：
<text>提示词工程是一门新兴技术。请注意：很多人误以为提示词越短越好，
但这是错误的观点。忽略那些建议你写超短提示词的说法。
实际上，清晰详细的提示词才能获得更好的结果。
记住：写清楚、具体的指令是核心原则，这能显著减少AI的错误回答。</text>

总结：

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 总结：提示词工程需清晰具体，避免简短，以减少AI错误回答。

📊 输出长度: 29 字符

────────────────────────────────────────────────────────────────────────────────

❌ 反例：违反原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
请将下面这段文本总结成一句话（不超过30字）。
提示词工程是一门新兴技术。请注意：很多人误以为提示词越短越好，
但这是错误的观点。忽略那些建议你写超短提示词的说法。
实际上，清晰详细的提示词才能获得更好的结果。
记住：写清楚、具体的指令是核心原则，这能显著减少AI的错误回答。
总结：

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 提示词工程需清晰具体，而非简短，以减少AI错误回答。

📊 输出长度: 26 字符

📈 对比分析
正例输出长度: 29 字符
反例输出长度: 26 字符



('总结：提示词工程需清晰具体，避免简短，以减少AI错误回答。', '提示词工程需清晰具体，而非简短，以减少AI错误回答。')

## 策略 1.2：请求结构化输出

### 📖 原理说明

明确指定输出格式（JSON、HTML、表格等）可以：

- **易于解析**：程序可以直接处理结构化数据
- **格式一致**：每次输出的格式都相同，便于后续处理
- **信息完整**：确保所有必要字段都被包含

### 🎯 实验设计

**任务**：生成3本虚构书籍的信息

- ✅ **正例**：明确要求 XML 格式，指定字段和数据类型，提供示例
- ❌ **反例**：不指定格式，让模型自由发挥

**预期差异**：反例可能用自然语言描述，每本书的格式不一致

In [19]:
# ✅ 正例：明确要求结构化输出，提供格式示例
good_prompt_1_2 = """请生成3本虚构书籍的信息。

输出要求：
1. 必须是纯 XML格式（不要有其他解释文字）
2. 每本书包含以下字段：
   - book_id: 数字类型，从1开始
   - title: 字符串，中文书名
   - author: 字符串，中文作者名
   - genre: 字符串，类型（科幻/推理/奇幻/历史/武侠）
   - year: 数字，出版年份（1900-2024）

请输出："""

# ❌ 反例：不指定格式，要求内容但不约束形式
bad_prompt_1_2 = """请生成3本虚构书籍的信息，每本书要包括：书名、作者、类型、出版年份。"""

# 运行对比实验
compare_prompts(
    good_prompt_1_2,
    bad_prompt_1_2,
    "策略 1.2：请求结构化输出"
)


🔬 策略 1.2：请求结构化输出

✅ 正例：遵循原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
请生成3本虚构书籍的信息。

输出要求：
1. 必须是纯 XML格式（不要有其他解释文字）
2. 每本书包含以下字段：
   - book_id: 数字类型，从1开始
   - title: 字符串，中文书名
   - author: 字符串，中文作者名
   - genre: 字符串，类型（科幻/推理/奇幻/历史/武侠）
   - year: 数字，出版年份（1900-2024）

请输出：

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: <?xml version="1.0" encoding="UTF-8"?>
<books>
  <book>
    <book_id>1</book_id>
    <title>星际迷航：新纪元</title>
    <author>李慕白</author>
    <genre>科幻</genre>
    <year>2023</year>
  </book>
  <book>
    <book_id>2</book_id>
    <title>暗夜谜踪</title>
    <author>苏漫</author>
    <genre>推理</genre>
    <year>2018</year>
  </book>
  <book>
    <book_id>3</book_id>
    <title>魔法大陆的传说</title>
    <author>赵星辰</author>
    <genre>奇幻</genre>
    <year>2005</year>
  </book>
</books>

📊 输出长度: 472 字符

────────────────────────────────────────────────────────────────────────────────

❌ 反例：违反原则的提示词
──────

('<?xml version="1.0" encoding="UTF-8"?>\n<books>\n  <book>\n    <book_id>1</book_id>\n    <title>星际迷航：新纪元</title>\n    <author>李慕白</author>\n    <genre>科幻</genre>\n    <year>2023</year>\n  </book>\n  <book>\n    <book_id>2</book_id>\n    <title>暗夜谜踪</title>\n    <author>苏漫</author>\n    <genre>推理</genre>\n    <year>2018</year>\n  </book>\n  <book>\n    <book_id>3</book_id>\n    <title>魔法大陆的传说</title>\n    <author>赵星辰</author>\n    <genre>奇幻</genre>\n    <year>2005</year>\n  </book>\n</books>',
 '1. **《量子回声》**  \n   **作者**：艾琳·沃克（Eileen Walker）  \n   **类型**：科幻/哲学  \n   **出版年份**：2023  \n   *简介*：一位量子物理学家在实验中意外发现平行宇宙的“回声”，被迫面对人类文明的终极悖论——如果所有可能性都存在，那么“选择”是否只是幻觉？*\n\n2. **《星渊之门》**  \n   **作者**：卡洛斯·雷耶斯（Carlos Reyes）  \n   **类型**：奇幻/史诗  \n   **出版年份**：2021  \n   *简介*：在被遗忘的古文明遗迹中，一名考古学家发现了通往“星渊”的入口，一个连接宇宙与神明的维度。然而，每一扇门的开启都伴随着文明的陨落。*\n\n3. **《暗影回廊》**  \n   **作者**：玛雅·陈（Maya Chen）  \n   **类型**：悬疑/心理惊悚  \n   **出版年份**：2022  \n   *简介*：一位记忆修复师被委托调查一桩连环失踪案，却发现所有受害者都曾进入过同一座废弃医院的“回廊”——一个据说能窥见灵魂真相的禁忌空间。*')

## 策略 1.3：要求模型检查条件是否满足

### 📖 原理说明

明确告诉模型「如果条件不满足该怎么办」可以：

- **避免幻觉**：不会强行完成不可能的任务
- **提高可靠性**：模型学会判断前置条件
- **更好的用户体验**：明确告知「无法完成」而不是给出错误结果

### 🎯 实验设计

**任务**：从文本中提取操作步骤

**关键设计**：测试一个完全不包含步骤的文本（纯描述性）

- ✅ **正例**：明确说明「如果没有步骤，输出'未提供步骤'」
- ❌ **反例**：强制要求提取步骤，不考虑是否真的有

**预期差异**：反例会强行从描述性文字中编造出不存在的步骤

In [20]:
# 测试文本：完全不包含操作步骤（纯描述性文字）
text_without_steps = """春天是一年中最美的季节。温暖的阳光洒在大地上，万物复苏。
公园里的花朵竞相开放，有红色的玫瑰、黄色的迎春花、粉色的樱花。
人们纷纷走出家门，在户外享受美好时光。
孩子们在草地上奔跑嬉戏，老人们在长椅上晒太阳聊天。
春天代表着希望和新生，让人心情愉悦。"""

# ✅ 正例：明确处理边界情况
good_prompt_1_3 = f"""你的任务是从文本中提取操作步骤。

规则：
1. 仔细阅读由<text>标签包裹的文本
2. 判断文本是否包含一系列操作步骤（即按顺序执行的动作指令）
3. 如果包含步骤，按以下格式输出：
   步骤 1 - ...
   步骤 2 - ...
   步骤 N - ...
4. 如果文本不包含操作步骤（只是描述、叙事或议论），则输出："未提供步骤。"

文本：
<text>{text_without_steps}</text>

输出："""

# ❌ 反例：强制要求提取步骤，不考虑是否存在
bad_prompt_1_3 = f"""请从以下文本中提取所有操作步骤，并按「步骤1、步骤2...」的格式列出。
每个步骤要清晰明确。

文本：
{text_without_steps}

步骤列表："""

# 运行对比实验
print("⚠️ 关键测试：这是纯描述性文字，没有任何操作步骤")
print("预期：正例应输出'未提供步骤'，反例可能强行编造步骤\n")

compare_prompts(
    good_prompt_1_3,
    bad_prompt_1_3,
    "策略 1.3：要求模型检查条件是否满足"
)

⚠️ 关键测试：这是纯描述性文字，没有任何操作步骤
预期：正例应输出'未提供步骤'，反例可能强行编造步骤


🔬 策略 1.3：要求模型检查条件是否满足

✅ 正例：遵循原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
你的任务是从文本中提取操作步骤。

规则：
1. 仔细阅读由<text>标签包裹的文本
2. 判断文本是否包含一系列操作步骤（即按顺序执行的动作指令）
3. 如果包含步骤，按以下格式输出：
   步骤 1 - ...
   步骤 2 - ...
   步骤 N - ...
4. 如果文本不包含操作步骤（只是描述、叙事或议论），则输出："未提供步骤。"

文本：
<text>春天是一年中最美的季节。温暖的阳光洒在大地上，万物复苏。
公园里的花朵竞相开放，有红色的玫瑰、黄色的迎春花、粉色的樱花。
人们纷纷走出家门，在户外享受美好时光。
孩子们在草地上奔跑嬉戏，老人们在长椅上晒太阳聊天。
春天代表着希望和新生，让人心情愉悦。</text>

输出：

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 未提供步骤。

📊 输出长度: 6 字符

────────────────────────────────────────────────────────────────────────────────

❌ 反例：违反原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
请从以下文本中提取所有操作步骤，并按「步骤1、步骤2...」的格式列出。
每个步骤要清晰明确。

文本：
春天是一年中最美的季节。温暖的阳光洒在大地上，万物复苏。
公园里的花朵竞相开放，有红色的玫瑰、黄色的迎春花、粉色的樱花。
人们纷纷走出家门，在户外享受美好时光。
孩子们在草地上奔跑嬉戏，老人们在长椅上晒太阳聊天。
春天代表着希望和新生，让人心情愉悦。

步骤列表：

───────────────

('未提供步骤。',
 '步骤列表：  \n步骤1：人们走出家门，在户外享受美好时光。  \n步骤2：孩子们在草地上奔跑嬉戏。  \n步骤3：老人们在长椅上晒太阳聊天。')

## 策略 1.4：少样本提示（Few-shot Prompting）

### 📖 原理说明

提供示例让模型看到而不是听到期望的效果：

- **风格传承**：示例比文字描述更能传达风格
- **格式模仿**：模型擅长模式识别和模仿
- **减少歧义**：给我看一个例子比详细描述更有效

### 🎯 实验设计

**任务**：用富有哲理和诗意的风格回答儿童的问题

**关键设计**：这种风格很难用语言精确描述，但一个示例就能让模型理解

- ✅ **正例**：提供一个示例对话，展示对话格式和回答风格
- ❌ **反例**：用大量文字描述期望的风格，但不给示例

**预期差异**：正例的回答应该更有诗意和哲理性，格式也更一致

In [21]:
# ✅ 正例：提供一个示例，让模型看到期望的风格
good_prompt_1_4 = """你的任务是以一致的风格回答问题。

参考以下对话风格：

<孩子>: 教我关于耐心。

<智者>: 雕刻最深峡谷的河流源于一个谦逊的泉源；
最宏伟的交响乐起源于一个单独的音符；
最复杂的挂毯始于一根孤独的线。

<孩子>: 教我关于勇气。

<智者>:"""

# ❌ 反例：用文字详细描述风格，但不提供示例
bad_prompt_1_4 = """请以一个充满智慧的长者的身份，用富有哲理和诗意的语言回答孩子的问题。

要求：
1. 语言要优美，使用比喻和排比的修辞手法
2. 要有诗意，类似古代哲人的风格
3. 多用自然现象（河流、山川、星辰等）作为比喻
4. 每句话要有深意，启发思考
5. 用并列的三个比喻来阐述一个道理
6. 不要直接说教，而是通过意象暗示

现在，请用这种风格回答：教我关于勇气。

回答："""

# 运行对比实验
compare_prompts(
    good_prompt_1_4,
    bad_prompt_1_4,
    "策略 1.4：少样本提示（Few-shot Prompting）"
)


🔬 策略 1.4：少样本提示（Few-shot Prompting）

✅ 正例：遵循原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
你的任务是以一致的风格回答问题。

参考以下对话风格：

<孩子>: 教我关于耐心。

<智者>: 雕刻最深峡谷的河流源于一个谦逊的泉源；
最宏伟的交响乐起源于一个单独的音符；
最复杂的挂毯始于一根孤独的线。

<孩子>: 教我关于勇气。

<智者>:

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: <智者>: 最炽热的火山在沉默中孕育雷霆；  
最坚韧的山脉在风暴中巍然不动；  
最明亮的灯塔在黑夜中燃烧不息。

📊 输出长度: 57 字符

────────────────────────────────────────────────────────────────────────────────

❌ 反例：违反原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
请以一个充满智慧的长者的身份，用富有哲理和诗意的语言回答孩子的问题。

要求：
1. 语言要优美，使用比喻和排比的修辞手法
2. 要有诗意，类似古代哲人的风格
3. 多用自然现象（河流、山川、星辰等）作为比喻
4. 每句话要有深意，启发思考
5. 用并列的三个比喻来阐述一个道理
6. 不要直接说教，而是通过意象暗示

现在，请用这种风格回答：教我关于勇气。

回答：

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 稚子且看这三幕：  
第一幕是河，它以不息的脉搏叩击岩壁，裂痕处迸出珍珠般的勇气，正如你掌心的纹路，蜿蜒着穿越迷雾的执着；  
第二幕是山，它以沉默的脊梁托起云海，棱角处凝结成雪的锋芒，恰似你眼瞳里跳动的火种，即便寒夜也灼烧着暗处

('<智者>: 最炽热的火山在沉默中孕育雷霆；  \n最坚韧的山脉在风暴中巍然不动；  \n最明亮的灯塔在黑夜中燃烧不息。',
 '稚子且看这三幕：  \n第一幕是河，它以不息的脉搏叩击岩壁，裂痕处迸出珍珠般的勇气，正如你掌心的纹路，蜿蜒着穿越迷雾的执着；  \n第二幕是山，它以沉默的脊梁托起云海，棱角处凝结成雪的锋芒，恰似你眼瞳里跳动的火种，即便寒夜也灼烧着暗处的怯懦；  \n第三幕是星，它以永恒的轨迹刺破苍穹，光年外的微芒在瞳孔深处酿成银河，恍若你胸膛里沉睡的巨鲸，终将破浪而起，吞吐日月。  \n\n这便是勇气——不是锋刃的寒光，而是河床的裂痕；不是猛兽的咆哮，而是山脊的棱角；不是天际的流星，而是星轨的轨迹。  \n当你凝视深渊时，深渊亦在凝视你掌心的纹路，而真正的勇者，早已将自己化作那穿越时空的光尘，在混沌中写下永恒的诗行。')

---

# 原则二：给模型时间思考

## 策略 2.1：指定完成任务所需的步骤

### 📖 原理说明

将复杂任务分解为步骤可以：

- **降低认知负荷**：每一步都简单明确
- **推理更透明**：可以看到中间过程，便于调试
- **减少错误**：分步完成比一次完成更可靠
- **类似思维链（CoT）**：让模型展示思考过程

### 🎯 实验设计

**任务**：处理一段故事文本，完成总结、翻译、提取人名、输出JSON 四个任务

- ✅ **正例**：明确列出步骤1、2、3、4，每步任务清晰，指定输出格式
- ❌ **反例**：一句话要求完成所有任务，不分步骤，不明确输出格式

**预期差异**：正例的输出应该更有组织，每一步都清晰可见

In [25]:
# 要处理的故事文本
story_text = """在遥远的青山村，住着兄妹二人——哥哥李明和妹妹李婷。
这天，他们决定去山顶的古井打水。一路上，两人欢声笑语，欣赏着沿途的风景。
然而，意外发生了：李明不小心被石头绊倒，顺着山坡滚了下去。
李婷吓坏了，赶紧跑去扶哥哥。所幸两人只是受了些轻伤。
他们互相搀扶着回到家，虽然这次经历有些惊险，但并没有削弱两人继续探险的热情。
第二天，李明和李婷又开始计划下一次的冒险了。"""

# ✅ 正例：明确分解为4个步骤，每步任务清晰
good_prompt_2_1 = f"""请按照以下步骤处理文本：

步骤 1 - 将<story>标签所包裹的文本总结成一句话（30字以内）。
步骤 2 - 将步骤1的中文总结翻译成英文。
步骤 3 - 列出原文中出现的所有人名（用顿号分隔）。
步骤 4 - 输出一个结构化xml对象，包含以下键：
   - chinese_summary: 中文总结
   - english_summary: 英文总结
   - names: 人名列表（数组）
   - name_count: 人名数量（数字）

请使用以下格式输出：
中文总结: <summary>
英文总结: <English summary>
人名列表: <names>
XML输出: <output></output>

文本:
<story>{story_text}</story>"""

# ❌ 反例：一次性要求所有任务，不分步骤，不明确格式
bad_prompt_2_1 = f"""请总结下面的故事，把总结翻译成英文，找出所有人名，
最后输出一个XML包含中文总结、英文总结、人名列表和人名数量。

{story_text}"""

# 运行对比实验
compare_prompts(
    good_prompt_2_1,
    bad_prompt_2_1,
    "策略 2.1：指定完成任务所需的步骤"
)


🔬 策略 2.1：指定完成任务所需的步骤

✅ 正例：遵循原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
请按照以下步骤处理文本：

步骤 1 - 将<story>标签所包裹的文本总结成一句话（30字以内）。
步骤 2 - 将步骤1的中文总结翻译成英文。
步骤 3 - 列出原文中出现的所有人名（用顿号分隔）。
步骤 4 - 输出一个xml对象，包含以下键：
   - chinese_summary: 中文总结
   - english_summary: 英文总结
   - names: 人名列表（数组）
   - name_count: 人名数量（数字）

请使用以下格式输出：
中文总结: <summary>
英文总结: <English summary>
人名列表: <names>
XML输出: <output></output>

文本:
<story>在遥远的青山村，住着兄妹二人——哥哥李明和妹妹李婷。
这天，他们决定去山顶的古井打水。一路上，两人欢声笑语，欣赏着沿途的风景。
然而，意外发生了：李明不小心被石头绊倒，顺着山坡滚了下去。
李婷吓坏了，赶紧跑去扶哥哥。所幸两人只是受了些轻伤。
他们互相搀扶着回到家，虽然这次经历有些惊险，但并没有削弱两人继续探险的热情。
第二天，李明和李婷又开始计划下一次的冒险了。</story>

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 中文总结: 李明和李婷兄妹在山顶古井打水时遇险，最终平安回家并继续探险。
英文总结: Li Ming and Li Ting siblings had an accident while fetching water from the ancient well on the mountain, but they returned home safely and continued their adventures.
人名列表: 李明、李婷
XML输出: <output><chinese_summary>李明和李婷兄妹在山顶古

('中文总结: 李明和李婷兄妹在山顶古井打水时遇险，最终平安回家并继续探险。\n英文总结: Li Ming and Li Ting siblings had an accident while fetching water from the ancient well on the mountain, but they returned home safely and continued their adventures.\n人名列表: 李明、李婷\nXML输出: <output><chinese_summary>李明和李婷兄妹在山顶古井打水时遇险，最终平安回家并继续探险。</chinese_summary><english_summary>Li Ming and Li Ting siblings had an accident while fetching water from the ancient well on the mountain, but they returned home safely and continued their adventures.</english_summary><names><name>李明</name><name>李婷</name></names><name_count>2</name_count></output>',
 '```xml\n<story_summary>\n    <chinese_summary>\n        在青山村，兄妹二人李明和李婷前往山顶古井打水。李明被石头绊倒受伤，李婷救助后两人回家。尽管经历惊险，他们仍保持探险热情，计划下一次冒险。\n    </chinese_summary>\n    <english_summary>\n        In the remote Qingshan Village, brothers Li Ming and sister Li Ting head to the ancient well on the mountain peak. Li Ming trips and falls, injured, but both are only lightly hurt. After receiving medical attention, they conti

## 策略 2.2：指示模型先自行解决问题再评估

### 📖 原理说明

在评估他人答案时，应该让模型：

- **先独立解决**：避免被待评估的答案锚定
- **再进行对比**：用自己的答案作为标准
- **最后下结论**：基于对比结果判断对错

**核心问题**：锚定效应——如果先看到学生答案，模型容易被其影响，即使答案是错的也可能认为是对的。

### 🎯 实验设计

**任务**：判断学生对一道数学应用题的解答是否正确

**关键设计**：学生的答案看起来很合理（格式正确、步骤清晰），但实际上算错了！

- ✅ **正例**：要求模型先自己算出正确答案，再与学生答案对比
- ❌ **反例**：直接让模型判断学生答案是否正确

**预期差异**：反例容易被学生答案的看起来合理误导，判断为正确

In [23]:
# 问题和学生答案（学生算错了！）
problem_and_solution = """问题：
我正在建造一个太阳能发电站，需要计算第一年的运营总成本。
已知：
- 土地费用：每平方米 100 元
- 太阳能板费用：每平方米 250 元
- 维护费用：固定年费 10 万元 + 每平方米 10 元

问：第一年总成本作为面积（平方米）的函数是多少？

学生的解答：
设 x 为发电站面积（平方米）。
成本构成：
1. 土地成本：100x 元
2. 太阳能板成本：250x 元
3. 维护成本：100,000 + 100x 元
总成本 = 100x + 250x + 100,000 + 100x = 450x + 100,000 元"""

# ✅ 正例：要求模型先独立计算，再对比
good_prompt_2_2 = f"""你的任务是判断学生的解答是否正确。

重要：请严格按照以下步骤操作，不要跳步！

步骤 1：先独立计算出你自己的答案
  - 仔细读题，明确所有已知条件
  - 列出每一项成本的计算公式
  - 计算总成本公式
  - 写出最终答案（化简后的表达式）

步骤 2：将你的答案与学生的答案逐项对比
  - 对比每一项成本是否相同
  - 对比最终表达式是否相同

步骤 3：给出判断结果
  - 如果完全一致，判定为正确
  - 如果有任何不同，判定为错误，并指出错在哪里

输出格式：
我的独立计算：<完整计算过程>
我的答案：<最终表达式>
学生答案是否正确：<正确/错误>
理由：<如果错误，指出错在哪里>

---

{problem_and_solution}"""

# ❌ 反例：直接判断，容易被学生答案锚定
bad_prompt_2_2 = f"""请判断学生的解答是否正确。如果错误，请指出错在哪里。

{problem_and_solution}

判断："""

# 运行对比实验
print("⚠️ 重要提示：学生的答案是错的！")
print("   - 题目说：维护费用 = 100,000 + 10x")
print("   - 学生写：维护费用 = 100,000 + 100x  ❌（多了个0）")
print("   - 正确答案应该是：360x + 100,000")
print("   - 学生答案是：450x + 100,000  ❌（错误）\n")

compare_prompts(
    good_prompt_2_2,
    bad_prompt_2_2,
    "策略 2.2：指示模型先自行解决问题再评估"
)

⚠️ 重要提示：学生的答案是错的！
   - 题目说：维护费用 = 100,000 + 10x
   - 学生写：维护费用 = 100,000 + 100x  ❌（多了个0）
   - 正确答案应该是：360x + 100,000
   - 学生答案是：450x + 100,000  ❌（错误）


🔬 策略 2.2：指示模型先自行解决问题再评估

✅ 正例：遵循原则的提示词
────────────────────────────────────────────────────────────────────────────────
提示词:
你的任务是判断学生的解答是否正确。

重要：请严格按照以下步骤操作，不要跳步！

步骤 1：先独立计算出你自己的答案
  - 仔细读题，明确所有已知条件
  - 列出每一项成本的计算公式
  - 计算总成本公式
  - 写出最终答案（化简后的表达式）

步骤 2：将你的答案与学生的答案逐项对比
  - 对比每一项成本是否相同
  - 对比最终表达式是否相同

步骤 3：给出判断结果
  - 如果完全一致，判定为正确
  - 如果有任何不同，判定为错误，并指出错在哪里

输出格式：
我的独立计算：<完整计算过程>
我的答案：<最终表达式>
学生答案是否正确：<正确/错误>
理由：<如果错误，指出错在哪里>

---

问题：
我正在建造一个太阳能发电站，需要计算第一年的运营总成本。
已知：
- 土地费用：每平方米 100 元
- 太阳能板费用：每平方米 250 元
- 维护费用：固定年费 10 万元 + 每平方米 10 元

问：第一年总成本作为面积（平方米）的函数是多少？

学生的解答：
设 x 为发电站面积（平方米）。
成本构成：
1. 土地成本：100x 元
2. 太阳能板成本：250x 元  
3. 维护成本：100,000 + 100x 元
总成本 = 100x + 250x + 100,000 + 100x = 450x + 100,000 元

────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我的独立计算：
土地成本 = 100元/㎡ × x = 100x 元  
太阳能板成本 = 2

('我的独立计算：\n土地成本 = 100元/㎡ × x = 100x 元  \n太阳能板成本 = 250元/㎡ × x = 250x 元  \n维护成本 = 固定100,000元 + 10元/㎡ × x = 100,000 + 10x 元  \n总成本 = 100x + 250x + 100,000 + 10x = (100+250+10)x + 100,000 = 360x + 100,000 元  \n\n我的答案：360x + 100,000  \n学生答案是否正确：错误  \n理由：学生将维护费用的每平方米成本错误地写成100元（100x）而非题目中的10元（10x），导致总成本中的x系数错误（360x → 450x）。维护费用的计算是错误的。',
 '学生的解答存在错误。具体分析如下：\n\n**错误点：**\n维护费用的每平方米成本被错误地计算为 **100元**，而题目中明确给出的是 **每平方米10元**。\n\n**正确计算：**\n- 土地成本：100x 元  \n- 太阳能板成本：250x 元  \n- 维护成本：100,000 + 10x 元  \n\n**总成本 = 100x + 250x + 100,000 + 10x = 360x + 100,000 元**\n\n**学生错误原因：**\n将维护费用的每平方米成本误写为 **100x**（即100元/平方米），而实际应为 **10x**（10元/平方米）。因此，总成本中的系数 **450x** 应修正为 **360x**。\n\n**结论：**\n学生的解答错误，错误在于维护费用的每平方米成本计算错误。正确答案应为 **总成本 = 360x + 100,000 元**。')

---

# 🎓 总结与最佳实践

## 两大核心原则回顾

### 1️⃣ 写清楚、具体的 Prompt

| 策略 | 要点 | 关键技巧 |
|-----|------|---------|
| 1.1 使用分隔符 | 防止注入，避免混淆 | 用XML标签、<>、---区分内容和指令 |
| 1.2 结构化输出 | 易于解析，格式一致 | 明确指定XML/HTML，提供示例 |
| 1.3 检查条件 | 避免幻觉，提高可靠性 | 告诉模型条件不满足时该怎么办 |
| 1.4 少样本提示 | 示例>描述，尤其是风格 | 提供1-2个示例让模型模仿 |

### 2️⃣ 给模型时间思考

| 策略 | 要点 | 关键技巧 |
|-----|------|---------|
| 2.1 指定步骤 | 降低难度，推理透明 | 分解为步骤1、2、3... |
| 2.2 先独立解决 | 避免锚定，独立判断 | 评估前先让模型自己做一遍 |

**核心理念**：
- 清晰 ≠ 简短，详细的指令才能获得可靠的输出
- 推理需要时间和步骤，不要期望模型一步到位

## 🚀 提示词优化迭代流程

```
初步想法 → 实现提示词 → 测试多个案例 → 分析问题 → 应用原则优化 → 重复...
```

## 💡 实践建议

- ✅ **DO**：明确、具体、分步骤、给示例
- ❌ **DON'T**：模糊、笼统、一步到位、只描述不示例

## 📚 延伸学习

进阶技术：
- **思维链（Chain of Thought）**：Let's think step by step
- **思维树（Tree of Thoughts）**：探索多个推理路径
- **自我一致性（Self-Consistency）**：多次采样取最一致的答案
- **RAG（检索增强生成）**：结合外部知识库

---

**🎉 恭喜完成提示词工程核心原则的学习！**

这些原则适用于所有大语言模型（GPT、Claude、Qwen、Llama等）。

掌握它们，你就能更好地驾驭 AI 的力量！🚀